In [111]:
import os

from distilabel.dataset import Dataset
import pandas as pd

# Original GPT4 response & label


In [191]:
data_dirs = [
    "/home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/fml-generation",
    "/home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/fml-generation-1",
    "/home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/dataset-6e09efb3-6f77-4422-949c-0c2b047cd17a/2024-02-09-15-31-49",
    "/home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/dataset-6e09efb3-6f77-4422-949c-0c2b047cd17a-micro",
]

In [192]:
dataset_dfs = []
for data_dir in data_dirs:
    for run_dir in os.listdir(data_dir):
        run_config = os.path.join(data_dir, run_dir, "config.yaml")
        checkpoints_dir = os.path.join(data_dir, run_dir, "checkpoints")
        if os.path.exists(run_config) and os.path.exists(checkpoints_dir):
            if "feedback" in os.listdir(checkpoints_dir):
                dataset = Dataset.load_from_disk(
                    os.path.join(checkpoints_dir, "feedback")
                )
                df = dataset.to_pandas()
                dataset_dfs.append(df)
                print(f"Dataset {checkpoints_dir} appended")

Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/fml-generation/2024-02-14-10-15-15/checkpoints appended
Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/fml-generation/2024-02-14-09-19-05/checkpoints appended
Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/fml-generation/2024-02-14-09-01-03/checkpoints appended
Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/fml-generation-1/2024-02-14-20-26-44/checkpoints appended
Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/fml-generation-1/2024-02-15-16-07-59/checkpoints appended
Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/fml-generation-1/2024-02-14-14-16-35/checkpoints appended
Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/fml-generation-1/2024

In [193]:
df = pd.concat(dataset_dfs)
df = df.dropna(subset=["rating"])
base_df = (
    df.explode("instructions")
    .explode("generations")
    .explode(["generations", "rating", "rationale"])
).explode("generation_model")
base_df

,index,domain,generation_model,generation_prompt,raw_generation_responses,function,parses,models,instructions,generations,__index_level_0__,labelling_model,labelling_prompt,raw_labelling_response,rating,rationale
0,0,Functions for search APIs,gpt-4-1106-preview,[[{'content': 'You are an AI assistant that th...,"[{\n ""function_calls"": [\n {\n ""name""...","{""function"":{""name"":""performSearch"",""descripti...",True,True,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...",0,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 2\nRationale: ...,2.0,The function call is correct and mostly comple...
0,0,Functions for search APIs,gpt-4-1106-preview,[[{'content': 'You are an AI assistant that th...,"[{\n ""function_calls"": [\n {\n ""name""...","{""function"":{""name"":""performSearch"",""descripti...",True,True,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...",0,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 2\nRationale: ...,2.0,The function call is correct and mostly comple...
0,0,Functions for search APIs,gpt-4-1106-preview,[[{'content': 'You are an AI assistant that th...,"[{\n ""function_calls"": [\n {\n ""name""...","{""function"":{""name"":""performSearch"",""descripti...",True,True,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...",0,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 2\nRationale: ...,3.0,"The function call is accurate, follows the ins..."
0,0,Functions for search APIs,gpt-4-1106-preview,[[{'content': 'You are an AI assistant that th...,"[{\n ""function_calls"": [\n {\n ""name""...","{""function"":{""name"":""performSearch"",""descripti...",True,True,Retrieve the latest articles regarding the eco...,"{""name"":""performSearch"",""arguments"":{""query"":""...",0,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 2\nRationale: ...,3.0,"The function call is accurate, follows the ins..."
0,0,Functions for search APIs,gpt-4-1106-preview,[[{'content': 'You are an AI assistant that th...,"[{\n ""function_calls"": [\n {\n ""name""...","{""function"":{""name"":""performSearch"",""descripti...",True,True,Show me documents on 'nanotechnology advanceme...,"{""name"":""performSearch"",""arguments"":{""query"":""...",0,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 2\nRationale: ...,2.0,The function call is correct and mostly comple...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,79,Functions for news APIs,gpt-4-1106-preview,[[{'content': 'You are an AI assistant that th...,"[{\n ""function_calls"": [\n {\n ...","{""function"":{""name"":""GetLatestNews"",""descripti...",True,True,Show me the top news stories in France today i...,"{""name"":""GetLatestNews"",""arguments"":{""source"":...",79,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 1\nRationale: ...,1.0,The function call does not capture the instruc...
181,79,Functions for news APIs,gpt-4-1106-preview,[[{'content': 'You are an AI assistant that th...,"[{\n ""function_calls"": [\n {\n ...","{""function"":{""name"":""GetLatestNews"",""descripti...",True,True,Retrieve articles related to renewable energy ...,"{""name"":""GetLatestNews"",""arguments"":{""source"":...",79,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 1\nRationale: ...,1.0,The function call provided is incorrect and in...
181,79,Functions for news APIs,gpt-4-1106-preview,[[{'content': 'You are an AI assistant that th...,"[{\n ""function_calls"": [\n {\n ...","{""function"":{""name"":""GetLatestNews"",""descripti...",True,True,Retrieve articles related to renewable energy ...,"{""name"":""GetLatestNews"",""arguments"":{""source"":

# Benchmark with feedback


In [212]:
data_dirs = [
    (
        "/home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary_benchmark_medium/data/fml-generation-1/",
        "functionary-medium-v2.2",
    ),
    (
        "/home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary-benchmark-generation/",
        "functionary-small-v2.2",
    ),
]

In [213]:
dataset = Dataset.load_from_disk(
    "/home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/benchmark_feedback_dataset",
)
_df = dataset.to_pandas()
_df["generation_model"] = "gpt-3.5-turbo-0125"

dataset = Dataset.load_from_disk(
    "/home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary_benchmark_medium_no_feedback",
)
__df = dataset.to_pandas()
__df["generation_model"] = "functionary-medium-v2.2"


dataset = Dataset.load_from_disk(
    "/home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary_benchmark_small_no_feedback",
)
___df = dataset.to_pandas()
___df["generation_model"] = "functionary-small-v2.2"

In [214]:
dataset_dfs = [_df, __df]
for data_dir, model in data_dirs:
    for run_dir in os.listdir(data_dir):
        run_config = os.path.join(data_dir, run_dir, "config.yaml")
        checkpoints_dir = os.path.join(data_dir, run_dir, "checkpoints")
        if os.path.exists(run_config) and os.path.exists(checkpoints_dir):
            if "benchmark_feedback" in os.listdir(checkpoints_dir):
                dataset = Dataset.load_from_disk(
                    os.path.join(checkpoints_dir, "benchmark_feedback")
                )
                df = dataset.to_pandas()
                df["generation_model"] = model
                dataset_dfs.append(df)
                print(f"Dataset {checkpoints_dir} appended")

Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary-benchmark-generation/2024-03-01-14-48-53/checkpoints appended
Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary-benchmark-generation/2024-03-01-11-12-08/checkpoints appended


In [185]:
import json

In [223]:
df = pd.concat(dataset_dfs)

In [224]:
df = df.dropna(subset=["rating"])
df["function"] = df["function"].apply(
    lambda f: f.replace("```json\n", "").replace("\n```", "")
)
df["function"] = df["function"].apply(lambda f: str(json.loads(f)))
benchmark_df = (
    df.explode("instructions")
    .explode("generations")
    .explode("generations")
    .explode(["rating", "rationale"])
)
benchmark_df

,function,instructions,generations,__index_level_0__,labelling_model,labelling_prompt,raw_labelling_response,rating,rationale,generation_model
0,"{'function': {'name': 'AddCalendarEvent', 'des...",Create a calendar event for my mom's birthday ...,"{""name"":""AddCalendarEvent"",""arguments"":{""title...",0.0,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 2\nRationale: ...,2.0,The function call provided in Response 1 is mo...,gpt-3.5-turbo-0125
0,"{'function': {'name': 'AddCalendarEvent', 'des...",Create a calendar event for my mom's birthday ...,"{""name"":""AddCalendarEvent"",""arguments"":{""title...",0.0,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 2\nRationale: ...,3.0,The function call in Response 2 is complete an...,gpt-3.5-turbo-0125
0,"{'function': {'name': 'AddCalendarEvent', 'des...",Create a calendar event for my mom's birthday ...,"{""name"":""AddCalendarEvent"",""arguments"":{""title...",0.0,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 2\nRationale: ...,2.0,The function call provided in Response 1 is mo...,gpt-3.5-turbo-0125
0,"{'function': {'name': 'AddCalendarEvent', 'des...",Create a calendar event for my mom's birthday ...,"{""name"":""AddCalendarEvent"",""arguments"":{""title...",0.0,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 2\nRationale: ...,3.0,The function call in Response 2 is complete an...,gpt-3.5-turbo-0125
0,"{'function': {'name': 'AddCalendarEvent', 'des...",Create a calendar event for my mom's birthday ...,"{""name"":""AddCalendarEvent"",""arguments"":{""title...",0.0,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 2\nRationale: ...,2.0,The function call provided in Response 1 is mo...,gpt-3.5-turbo-0125
...,...,...,...,...,...,...,...,...,...,...
457,"{'function': {'name': 'GetLatestNews', 'descri...",Show me the recent articles in the technology ...,"{""name"":""GetLatestNews"",""arguments"":{""category...",NaN,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 3\nRationale: ...,3.0,The function call is correct as it accurately ...,functionary-small-v2.2
458,"{'function': {'name': 'GetLatestNews', 'descri...",Update me on the latest sports news in Spanish...,"{""name"":""GetLatestNews"",""arguments"":{""category...",NaN,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 3\nRationale: ...,3.0,The function call is correct and relevant to t...,functionary-small-v2.2
459,"{'function': {'name': 'GetLatestScores', 'desc...",Could you provide the score updates for the NH...,"{""name"":""GetLatestScores"",""arguments"":{""sportT...",NaN,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 3\nRationale: ...,3.0,The function call is entirely correct. It adhe...,functionary-small-v2.2
0,"{'function': {'name': 'AddCalendarEvent', 'des...",Create a calendar event for my mom's birthday ...,"{""name"":""AddCalendarEvent"",""arguments"":{""title...",NaN,gpt-4,[{'content': 'Your role is to evaluate functio...,#### Output for Text 1\nRating: 3\nRationale: ...,3.0,"The function call is accurate and complete, pr...",functionary-small-v2.2


In [225]:
benchmark_df.generation_model.value_counts()

functionary-medium-v2.2    7112
gpt-3.5-turbo-0125         6476
functionary-small-v2.2      636
Name: generation_model, dtype: int64

In [34]:
dataset = Dataset.from_pandas(df)

In [36]:
dataset.save_to_disk(
    "/home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary_benchmark_small"
)

Saving the dataset (1/1 shards): 100%|██████████| 1271/1271 [00:00<00:00, 106533.98 examples/s]


# benchmark no feedback


In [98]:
data_dirs = [
    (
        "/home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary-benchmark-generation/",
        "functionary-small-v2.2",
    ),
]

In [99]:
dataset_dfs = []
for data_dir, model in data_dirs:
    for run_dir in os.listdir(data_dir):
        run_config = os.path.join(data_dir, run_dir, "config.yaml")
        checkpoints_dir = os.path.join(data_dir, run_dir, "checkpoints")
        if os.path.exists(run_config) and os.path.exists(checkpoints_dir):
            if "benchmark_generate" in os.listdir(checkpoints_dir):
                dataset = Dataset.load_from_disk(
                    os.path.join(checkpoints_dir, "benchmark_generate")
                )
                df = dataset.to_pandas()
                df["generation_model"] = model
                dataset_dfs.append(df)
                print(f"Dataset {checkpoints_dir} appended")

Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary-benchmark-generation/2024-03-01-11-03-32/checkpoints appended
Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary-benchmark-generation/2024-03-01-11-04-53/checkpoints appended
Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary-benchmark-generation/2024-03-01-10-29-06/checkpoints appended
Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary-benchmark-generation/2024-03-01-11-00-19/checkpoints appended
Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary-benchmark-generation/2024-03-01-11-07-29/checkpoints appended
Dataset /home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary-benchmark-generation/2024-03-01-11-05-43/checkpoints appended
Dataset /h

In [100]:
df = pd.concat(dataset_dfs)
# df = df.dropna(subset=["rating"])
df = df.dropna(subset=["function_call"])
df = df.explode("instructions").explode("generations").explode("generations")
df = df.drop_duplicates(subset=["instructions", "generations", "function"])
df

,function,instructions,generations,generation_model,generation_prompt,raw_generation_responses,function_call
0,"```json\n{\n ""function"": {\n ""name"":...",Create a calendar event for my mom's birthday ...,"```json\n{\n ""name"": ""AddCalendarEvent"",\n ...",functionary-small-v2.2,[[{'content': 'Create a calendar event for my ...,[None],"[{""function_calls"": {""id"": ""call_KLZkdgi7qKssl..."
1,"```json\n{\n ""function"": {\n ""name"":...",Organize a surprise birthday party for John Do...,"```json\n{\n ""name"": ""AddCalendarEvent"",\n ...",functionary-small-v2.2,[[{'content': 'Organize a surprise birthday pa...,[None],"[{""function_calls"": {""id"": ""call_ZohBA8QfL3q3X..."
2,"```json\n{\n ""function"": {\n ""name"":...",Schedule a team meeting for next Monday at 3 P...,"```json\n{\n ""name"": ""AddCalendarEvent"",\n ...",functionary-small-v2.2,[[{'content': 'Schedule a team meeting for nex...,[None],"[{""function_calls"": {""id"": ""call_uPJzjc45f6cOw..."
3,"```json\n{\n ""function"": {\n ""name"":...",Set a reminder for my dentist appointment on t...,"```json\n{\n ""name"": ""AddCalendarEvent"",\n ...",functionary-small-v2.2,[[{'content': 'Set a reminder for my dentist a...,[None],"[{""function_calls"": {""id"": ""call_ATYn9GgZWNx24..."
4,"```json\n{\n ""function"": {\n ""name"":...",Could you arrange a birthday celebration for S...,"```json\n{\n ""name"": ""AddEvent"",\n ""argu...",functionary-small-v2.2,[[{'content': 'Could you arrange a birthday ce...,[None],"[{""function_calls"": {""id"": ""call_egzEj6uoipicT..."
...,...,...,...,...,...,...,...
1559,"```json\n{\n ""function"": {\n ""name"":...",Retrieve the latest blog post titles from www....,"```json\n{\n ""name"": ""extractWebData"",\n ...",functionary-small-v2.2,[[{'content': 'Retrieve the latest blog post t...,[None],"[{""function_calls"": [{""name"": ""extract"", ""argu..."
1656,"```json\n{\n ""function"": {\n ""name"":...","With these quarterly performance figures, can ...","```json\n{\n ""name"": ""generateBarChart"",\n ...",functionary-small-v2.2,[[{'content': 'With these quarterly performanc...,[None],"[{""function_calls"": [{""name"": ""generate"", ""arg..."
41,"```json\n{\n ""function"": {\n ""name"":...",I've made changes to the index.py script; woul...,"```json\n{\n ""name"": ""CodeAnalyzer"",\n ""...",functionary-small-v2.2,[[{'content': 'I've made changes to the index....,[None],"[{""function_calls"": [{""name"": ""Code"", ""argumen..."
191,"```json\n{\n ""function"": {\n ""name"":...","Please jot down a memo titled ""Project X Strat...","```json\n{\n ""name"": ""CreateNote"",\n ""ar...",functionary-small-v2.2,"[[{'content': 'Please jot down a memo titled ""...",[None],"[{""function_calls"": [{""name"": ""Create"", ""argum..."


In [101]:
pd.concat(dataset_dfs)

,function,instructions,generations,generation_model,generation_prompt,raw_generation_responses,function_call
0,"```json\n{\n ""function"": {\n ""name"":...",[Create a calendar event for my mom's birthday...,"[[```json\n{\n ""name"": ""AddCalendarEvent"",\...",functionary-small-v2.2,[[{'content': 'Create a calendar event for my ...,[None],"[{""function_calls"": {""id"": ""call_KLZkdgi7qKssl..."
1,"```json\n{\n ""function"": {\n ""name"":...",[Organize a surprise birthday party for John D...,"[[```json\n{\n ""name"": ""AddCalendarEvent"",\...",functionary-small-v2.2,[[{'content': 'Organize a surprise birthday pa...,[None],"[{""function_calls"": {""id"": ""call_ZohBA8QfL3q3X..."
2,"```json\n{\n ""function"": {\n ""name"":...",[Schedule a team meeting for next Monday at 3 ...,"[[```json\n{\n ""name"": ""AddCalendarEvent"",\...",functionary-small-v2.2,[[{'content': 'Schedule a team meeting for nex...,[None],"[{""function_calls"": {""id"": ""call_uPJzjc45f6cOw..."
3,"```json\n{\n ""function"": {\n ""name"":...",[Set a reminder for my dentist appointment on ...,"[[```json\n{\n ""name"": ""AddCalendarEvent"",\...",functionary-small-v2.2,[[{'content': 'Set a reminder for my dentist a...,[None],"[{""function_calls"": {""id"": ""call_ATYn9GgZWNx24..."
4,"```json\n{\n ""function"": {\n ""name"":...",[Could you arrange a birthday celebration for ...,"[[```json\n{\n ""name"": ""AddEvent"",\n ""ar...",functionary-small-v2.2,[[{'content': 'Could you arrange a birthday ce...,[None],"[{""function_calls"": {""id"": ""call_egzEj6uoipicT..."
...,...,...,...,...,...,...,...
7,"```json\n{\n ""function"": {\n ""name"":...","[Schedule a meeting called ""Budget Review"" in ...","[[```json\n{\n ""name"": ""AddEvent"",\n ""ar...",functionary-small-v2.2,"[[{'content': 'Schedule a meeting called ""Budg...","[None, None, None, None]","[{""id"":""call_UyoDPMGJ1xlBOZaLA8MoswGu"",""functi..."
8,"```json\n{\n ""function"": {\n ""name"":...","[Set up a dinner appointment titled ""Anniversa...","[[```json\n{\n ""name"": ""AddEvent"",\n ""ar...",functionary-small-v2.2,[[{'content': 'Set up a dinner appointment tit...,"[None, None, None, None]","[{""id"":""call_8JzNrnfBbIETVvV6amNs1W34"",""functi..."
9,"```json\n{\n ""function"": {\n ""name"":...",[Could you ban the user with ID '12345abc' for...,"[[```json\n{\n ""name"": ""BanUser"",\n ""arg...",functionary-small-v2.2,[[{'content': 'Could you ban the user with ID ...,"[None, None]","[{""id"":""call_FzJaOBOsRFjzR0B69zT65X1L"",""functi..."
0,"```json\n{\n ""function"": {\n ""name"":...",[Create a calendar event for my mom's birthday...,"[[```json\n{\n ""name"": ""AddCalendarEvent"",\...",functionary-small-v2.2,[[{'content': 'Create a calendar event for my ...,[None],"[{""function_calls"": [{""name"": ""AddCalendarEven..."


In [108]:
small_no_feedback_df = (
    df.groupby(["function", "instructions"])
    .aggregate(
        {
            "function_call": lambda fc: [[f] for f in fc],
            "generation_model": "first",
        }
    )
    .reset_index()
)
small_no_feedback_df = small_no_feedback_df.explode("function_call")
small_no_feedback_df["generations"] = small_no_feedback_df["function_call"]
small_no_feedback_df["instructions"] = small_no_feedback_df["instructions"].apply(
    lambda i: [i]
)
small_no_feedback_dataset = Dataset.from_pandas(small_no_feedback_df)

In [110]:
small_no_feedback_dataset.save_to_disk(
    "/home/ben/code/distilabel-workbench-function/scripts/function_calling_dataset/data/functionary_benchmark_small_no_feedback"
)

Saving the dataset (1/1 shards): 100%|██████████| 1569/1569 [00:00<00:00, 257638.61 examples/s]


In [109]:
small_no_feedback_df

,function,instructions,function_call,generation_model,generations
0,"```json\n{\n ""function"": {\n ""name"":...",[Create a calendar event for my mom's birthday...,"[[{""function_calls"": {""id"": ""call_KLZkdgi7qKss...",functionary-small-v2.2,"[[{""function_calls"": {""id"": ""call_KLZkdgi7qKss..."
1,"```json\n{\n ""function"": {\n ""name"":...",[Organize a surprise birthday party for John D...,"[[{""function_calls"": {""id"": ""call_ZohBA8QfL3q3...",functionary-small-v2.2,"[[{""function_calls"": {""id"": ""call_ZohBA8QfL3q3..."
2,"```json\n{\n ""function"": {\n ""name"":...",[Schedule a team meeting for next Monday at 3 ...,"[[{""function_calls"": {""id"": ""call_uPJzjc45f6cO...",functionary-small-v2.2,"[[{""function_calls"": {""id"": ""call_uPJzjc45f6cO..."
3,"```json\n{\n ""function"": {\n ""name"":...",[Set a reminder for my dentist appointment on ...,"[[{""function_calls"": {""id"": ""call_ATYn9GgZWNx2...",functionary-small-v2.2,"[[{""function_calls"": {""id"": ""call_ATYn9GgZWNx2..."
4,"```json\n{\n ""function"": {\n ""name"":...",[Could you arrange a birthday celebration for ...,"[[{""function_calls"": {""id"": ""call_egzEj6uoipic...",functionary-small-v2.2,"[[{""function_calls"": {""id"": ""call_egzEj6uoipic..."
...,...,...,...,...,...
1476,"```json\n{\n ""function"": {\n ""name"":...",[Please move any emails containing the phrase ...,"[[{""function_calls"": [{""name"": ""filterEmails"",...",functionary-small-v2.2,"[[{""function_calls"": [{""name"": ""filterEmails"",..."
1477,"```json\n{\n ""function"": {\n ""name"":...",[Show me emails from [email protected] that co...,"[[{""function_calls"": [{""name"": ""filterEmails"",...",functionary-small-v2.2,"[[{""function_calls"": [{""name"": ""filterEmails"",..."
1478,"```json\n{\n ""function"": {\n ""name"":...","[Using the keywords 'project update', how can ...","[[{""function_calls"": [{""name"": ""filterEmails"",...",functionary-small-v2.2,"[[{""function_calls"": [{""name"": ""filterEmails"",..."
1479,"```json\n{\n ""function"": {\n ""name"":...",[Provide a visualization of this quarter's sal...,"[[{""function_calls"": [{""name"": ""generate"", ""ar...",functionary-small-v2.2,"[[{""function_calls"": [{""name"": ""generate"", ""ar..."


# concatenat and wrangle


base_df


In [194]:
base_df = base_df[
    [
        "function",
        "instructions",
        "generations",
        "rating",
        "rationale",
        "generation_model",
    ]
].rename(columns={"generations": "function_call"})

In [226]:
benchmark_df = benchmark_df[
    [
        "function",
        "instructions",
        "generations",
        "rating",
        "rationale",
        "generation_model",
    ]
].rename(columns={"generations": "function_call"})

In [229]:
df = pd.concat([base_df, benchmark_df])

In [230]:
df = (
    df.groupby(["function", "instructions", "function_call", "generation_model"])
    .aggregate(
        {
            "rating": "mean",
            "rationale": lambda r: ", ".join(r),
        }
    )
    .reset_index()
)

In [231]:
from ast import literal_eval


def try_json(s):
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        return literal_eval(s)
    except TypeError:
        return None


df["function"] = df["function"].apply(try_json)

In [232]:
df.loc[df["function"].isna(), "rating"] = 0
df.loc[df["function"].isna(), "rationale"] = "Does not parse"

In [233]:
df.rating.value_counts()

3.000000    7533
1.000000    3008
2.000000    2211
2.500000    1515
1.500000      93
2.600000      27
2.750000      16
1.666667      10
2.200000       9
1.250000       6
2.666667       5
2.333333       3
2.250000       2
2.625000       2
2.900000       2
1.333333       1
2.800000       1
2.400000       1
Name: rating, dtype: int64

In [208]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

In [234]:
df.generation_model.value_counts()

gpt-4-1106-preview         10307
functionary-medium-v2.2     2033
gpt-3.5-turbo-0125          1620
functionary-small-v2.2       485
Name: generation_model, dtype: int64

In [235]:
dataset.push_to_hub("burtenshaw/function_calling_dataset")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.01it/s]
README.md: 100%|██████████| 15.9k/15.9k [00:00<00:00, 3.77MB/s]


CommitInfo(commit_url='https://huggingface.co/datasets/burtenshaw/function_calling_dataset/commit/535946a39f119f11feb4ceb390a177be292db7bc', commit_message='Upload dataset', commit_description='', oid='535946a39f119f11feb4ceb390a177be292db7bc', pr_url=None, pr_revision=None, pr_num=None)

In [247]:
dpo_dataset = []
df["function"] = df["function"].apply(lambda f: str(f))
for (function, instruction), _df in df.groupby(["function", "instructions"]):
    min_rating = _df.rating.min()
    max_rating = _df.rating.max()

    if min_rating == max_rating:
        continue

    chosen = _df.sort_values("rating", ascending=False).iloc[0]
    chosen_rating = chosen.rating
    chosen_rationale = chosen.rationale
    chosen_function_call = chosen.function_call

    rejected = _df.sort_values("rating", ascending=False).iloc[-1]
    rejected_rating = rejected.rating
    rejected_rationale = rejected.rationale
    rejected_function_call = rejected.function_call

    dpo_dataset.append(
        {
            "function": function,
            "instruction": instruction,
            "chosen_rating": chosen_rating,
            "chosen_rationale": chosen_rationale,
            "chosen": chosen_function_call,
            "rejected_rating": rejected_rating,
            "rejected_rationale": rejected_rationale,
            "rejected": rejected_function_call,
        }
    )

In [248]:
dpo_df = pd.DataFrame(dpo_dataset)

In [249]:
dpo_df

,function,instruction,chosen_rating,chosen_rationale,chosen,rejected_rating,rejected_rationale,rejected
0,"{'function': {'name': 'AddCalendarEvent', 'des...",Create a calendar event for my mom's birthday ...,3.0,The response correctly calls the `AddCalendarE...,"{""name"":""AddCalendarEvent"",""arguments"":{""title...",2.0,While the function call is complete and relate...,"{""name"":""AddCalendarEvent"",""arguments"":{""title..."
1,"{'function': {'name': 'AddCalendarEvent', 'des...",Include the team meeting scheduled for next Th...,3.0,"The function call is complete and accurate, fo...","{""name"":""AddCalendarEvent"",""arguments"":{""title...",2.0,While the function call is complete and correl...,"{""name"":""AddCalendarEvent"",""arguments"":{""title..."
2,"{'function': {'name': 'AddCalendarEvent', 'des...","Register a webinar titled ""Future of AI"" on my...",2.0,"The function call is correct, follows the inst...","{""name"":""AddCalendarEvent"",""arguments"":{""title...",1.0,The response does not correctly represent the ...,"{""name"":""AddCalendarEvent"",""arguments"":{""title..."
3,"{'function': {'name': 'AddCalendarEvent', 'des...",Schedule a team meeting for next Monday at 3 P...,3.0,"The function call is completely accurate, foll...","{""name"":""AddCalendarEvent"",""arguments"":{""title...",2.0,The function call is mostly complete and relev...,"{""name"":""AddCalendarEvent"",""arguments"":{""title..."
4,"{'function': {'name': 'AddCalendarEvent', 'des...","Schedule a webinar titled ""AI Innovations"" on ...",3.0,The response correctly calls the `AddCalendarE...,"{""name"":""AddCalendarEvent"",""arguments"":{""title...",2.0,The function call is accurate and follows the ...,"{""name"":""AddCalendarEvent"",""arguments"":{""title..."
...,...,...,...,...,...,...,...,...
2097,"{'title': 'Tool', 'type': 'object', 'required'...",Walk me through the steps necessary to export ...,3.0,The function call perfectly fits the instructi...,"{""name"":""export_calendar_events"",""arguments"":{...",2.0,The function call is accurate to the instructi...,"{""name"":""check_unread_emails"",""arguments"":{}}"
2098,"{'tool': {'function': {'name': 'FilterSpam', '...",I'd like assistance with ensuring that my spam...,3.0,The function call is complete and follows the ...,"{""name"":""FilterSpam"",""arguments"":{""threshold"":...",1.0,The function call does not follow the instruct...,"{""name"":""FilterSpam"",""arguments"":{""threshold"":..."
2099,"{'tool': {'function': {'name': 'FilterSpam', '...",I've been receiving too many unwanted newslett...,3.0,"The function call is correct, following the pr...","{""name"":""FilterSpam"",""arguments"":{""threshold"":...",2.0,The function call seems to follow the instruct...,"{""name"":""FilterSpam"",""arguments"":{""customRules..."
2100,"{'tool': {'function': {'name': 'FilterSpam', '...",Is it possible to tailor the junk mail filteri...,3.0,The function call is complete and follows the ...,"{""name"":""FilterSpam"",""arguments"":{""threshold"":...",1.0,The function call does not follow the instruct...,"{""name"":""FilterSpam"",""arguments"":{""threshold"":..."


In [250]:
dpo_dataset = Dataset.from_pandas(dpo_df)

dpo_dataset.push_to_hub("burtenshaw/function_calling_dpo")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.76s/it]
README.md: 100%|██████████| 28.0/28.0 [00:00<00:00, 168kB/s]


CommitInfo(commit_url='https://huggingface.co/datasets/burtenshaw/function_calling_dpo/commit/fefd57234e7db5c7870eafac476db926b35949ed', commit_message='Upload dataset', commit_description='', oid='fefd57234e7db5c7870eafac476db926b35949ed', pr_url=None, pr_revision=None, pr_num=None)